# Contract Analysis System with CrewAI

### Usage Instructions
- Place your contract PDF in the notebook directory
- Set up .env file with OpenAI API key
- un all cells in sequence
- View the boxen-formatted output for:
  - Agent profiles
  - Task details
  - Analysis results

### Overview Model Structure :

- https://drive.google.com/file/d/1JHwZSvCJ6DwIW7yWb8H74osdPhbqqnOn/view?usp=drive_link


### Setup and Dependencies

In [ ]:
# Install required packages
! pip install crewai pyboxen langchain python-dotenv PyPDF2 agentops openai


# Setting Up AI Agent Environment

## Required Imports

First, we need to import all necessary libraries for our AI agent workflow:

In [10]:
# Import necessary libraries and modules

# Load environment variables from .env file
from dotenv import load_dotenv

# Operating system interface
import os

# CrewAI components for agent orchestration
from crewai import Crew, Agent

# Pretty printing tool for terminal output
from pyboxen import boxen

# Tool for monitoring and logging AI agent operations
import agentops

# LangChain's ChatOpenAI model integration
from langchain.chat_models import ChatOpenAI

# Initialize environment variables from .env file
# This will look for a .env file in the current directory
# and load any variables defined there into the environment
load_dotenv()

True

# Environment and Monitoring Setup

#### Required Environment Variables Create an ENV file and add all the keys there:

- OPENAI_API_KEY=your-openai-api-key
- Get API key: https://docs.agentops.ai/v1/quickstart
- AGENTOPS_API_KEY=your-agentops-api-key
- AGENTOPS_PARENT_KEY=your-agentops-parent-key

#### Monitoring & Logs
- To monitor agent activity and view logs in agentop ai dashboard:
- Run the code to initiate the agent session.
- The session will generate a unique URL. You can use this link to access the AgentOps AI dashboard and view detailed logs and flow of the agents like below :

- Session link: https://app.agentops.ai/drilldown?session_id=<session-id>
- Example: 2024-12-24 19:53:24,323 - INFO: View info at https://app.agentops.ai/drilldown?session_id=8d76b2ea-...

In [11]:
# Initialize AgentOps for monitoring
agentops.init(tags=['contract-analysis'])

# Load OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

2024-12-24 19:53:24,323 - 22992 - client.py-client:258 - INFO: View info on this session at https://app.agentops.ai/drilldown?session_id=8d76b2ea-de2f-424b-afe8-afd267d95b6b


## Reference Data for PDF Extraction Comparison

This document serves as the reference for comparing the data extracted from the PDF. The data extracted from the PDF will be compared with the information provided below to ensure accuracy and completeness.

In [8]:
reference_data = {
    "limitation of liability": "30 days",
    "owner expiry date": "30 days",
    "notice period": "30 days",
    "agreement duration": "12 months",
    "payment terms": "30 days",
    "confidentiality period": "3 years",
    "insurance coverage": "$1,000,000",
    "maximum monthly hours": "120"
}

#### Showing Reference Data Log

In [13]:
print(boxen(
    "\n".join([f"{k}: {v}" for k, v in reference_data.items()]),
    title="📋 Reference Contract Terms",
    padding=1,
    margin=1,
    color="blue"
))

                                                                                                                   
   ╭─ 📋 Reference Contract Terms ────────╮                                                                        
   │                                      │                                                                        
   │   limitation of liability: 30 days   │                                                                        
   │   owner expiry date: 30 days         │                                                                        
   │   notice period: 30 days             │                                                                        
   │   agreement duration: 12 months      │                                                                        
   │   payment terms: 30 days             │                                                                        
   │   confidentiality period: 3 years    │                              

## PDF Reading and Reference Data Access Tools

### Overview

This code defines two tools: one for reading and extracting text from PDF files, and another for accessing reference data to compare the extracted content.

### Tools

#### 1. **PdfReadTool**: 
This tool is designed to extract text from a PDF file. It utilizes the `PyPDF2` library to read PDF files and extract text from each page.

- **Name**: `PDF Reader`
- **Description**: Reads and extracts text from PDF files.
- **Functionality**: 
  - Opens a PDF file provided by the `file_path` argument.
  - Extracts text from each page of the PDF.
  - Returns the extracted text or an error message if something goes wrong.

#### 2. **ReferenceDataTool**: 
This tool provides access to standard reference data that can be used for comparison against the data extracted from PDFs.

- **Name**: `Reference Data Tool`
- **Description**: Accesses the standard reference data for comparison.
- **Functionality**: 
  - Returns predefined reference data (in the `reference_data` variable) for comparison with the PDF content.
  - This reference data can be used to verify the accuracy of the extracted PDF data.

---

### Purpose

These tools are part of a larger system for reading PDF files and comparing their contents with standard reference data to ensure the accuracy and consistency of extracted information.


In [14]:
# Import required tools and PDF processing libraries
from langchain_community.tools import BaseTool
from PyPDF2 import PdfReader

# Custom PDF Reading Tool
class PdfReadTool(BaseTool):
    """Tool for reading and extracting text from PDF files"""
    name = "PDF Reader"
    description = "Read PDF files"

    def _run(self, file_path: str) -> str:
        try:
            # Open and process the PDF file
            with open(file_path, 'rb') as file:
                reader = PdfReader(file)
                text = ""
                # Extract text from each page
                for page in reader.pages:
                    text += page.extract_text()
            return text
        except Exception as e:
            return f"Error: {str(e)}"

# Reference Data Access Tool        
class ReferenceDataTool(BaseTool):
    """Tool for accessing standard reference data"""
    name = "Reference Data Tool"
    description = "Access the standard reference data for comparison"

    def _run(self, action_input: str = None) -> dict:
        return reference_data

## Memory System Components and Configuration

### Purpose

This Python script uses the `rich` library to create and display a table of memory system components, followed by a configuration code example that demonstrates how to implement these memory components in a system.

### Key Features

1. **Memory System Components Table**:
   - The script generates a table that lists and describes various memory components used in an AI system.
   - The components include:
     - **Short-Term Memory**: Temporarily stores recent interactions using Retrieval Augmented Generation (RAG).
     - **Long-Term Memory**: Preserves valuable insights over time to improve agent knowledge.
     - **Entity Memory**: Stores information about entities like people and places, enhancing agent's understanding.
     - **Contextual Memory**: Combines short-term, long-term, and entity memory for coherent agent responses.
     - **User Memory**: Stores user-specific information for better personalization.

2. **Code Configuration Example**:
   - The script also shows a code snippet that demonstrates how to configure the memory system using the `CrewAI` library. It configures:
     - **Long-term Memory**: Uses SQLite storage.
     - **Short-term Memory**: Uses RAG-based storage for temporary data.
     - **Entity Memory**: Stores information about entities using a custom RAG storage system.
   - The configuration includes parameters for setting up agents, tasks, and the process workflow, along with enabling detailed logging.

### How It Works

- **Table Creation**: The `rich` library is used to create a table with `box.DOUBLE` style and bold cyan headers, displaying the memory components and their descriptions.
- **Code Example**: The code snippet is displayed with syntax highlighting using the `Syntax` feature from `rich`, which helps to present the configuration in a visually appealing manner.

---

### Libraries Used:
- `rich`: For creating styled tables and syntax highlighting.
- `PyPDF2` (assumed context): Though not explicitly mentioned in this script, it might be used elsewhere for reading PDF files (as per previous context).

This script is part of a system that configures AI agents with memory capabilities for handling complex tasks and maintaining context over time.


In [7]:
from rich import box
from rich.console import Console
from rich.table import Table
from rich.syntax import Syntax

def create_tables():
    console = Console()
    
    # Memory Components Table: Creating a table to represent different memory components in the system
    components_table = Table(box=box.DOUBLE, title="Memory System Components", 
                           show_header=True, header_style="bold cyan", show_lines=True)
    
    # Defining columns for the table: one for component names and one for descriptions
    components_table.add_column("Component", style="bold white")
    components_table.add_column("Description", style="dim white")
    
    # List of components with corresponding descriptions
    components = [
        ("Short-Term Memory", "Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall and utilize information relevant to their current context during executions."),
        ("Long-Term Memory", "Preserves valuable insights and learnings from past executions, allowing agents to build and refine their knowledge over time."),
        ("Entity Memory", "Captures and organizes information about entities (people, places, concepts) encountered during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for storing entity information."),
        ("Contextual Memory", "Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and EntityMemory, aiding in coherence and relevance of agent responses."),
        ("User Memory", "Stores user-specific information and preferences, enhancing personalization and user experience.")
    ]
    
    # Adding rows for each component and its description
    for component, desc in components:
        components_table.add_row(component, desc)
    
    # Code Configuration Example: Showing a code snippet that demonstrates how to configure the memory system
    code_example = '''
from crewai import Crew, Agent, Task, Process

my_crew = Crew(
    agents=[...],  # List of AI agents that will work on tasks
    tasks=[...],   # List of tasks to be executed
    process="Process.sequential",  # Tasks will be executed in sequence
    memory=True,   # Enable memory system
    
    # Long-term memory configuration
    long_term_memory=EnhanceLongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="/my_data_dir/my_crew1/long_term_memory_storage.db"
        )
    ),
    
    # Short-term memory configuration
    short_term_memory=EnhanceShortTermMemory(
        storage=CustomRAGStorage(
            crew_name="my_crew",           # Unique identifier for the crew
            storage_type="short_term",     # Specifies memory type as short-term
            data_dir="//my_data_dir",      # Directory for storing memory data
            model=embedder["model"],       # Embedding model for RAG
            dimension=embedder["dimension"],# Dimension of embeddings
        ),
    ),
    
    # Entity memory configuration
    entity_memory=EnhanceEntityMemory(
        storage=CustomRAGStorage(
            crew_name="my_crew",           # Same crew identifier
            storage_type="entities",       # Entity-specific storage
            data_dir="//my_data_dir",      # Same data directory
            model=embedder["model"],       # Same embedding model
            dimension=embedder["dimension"],# Same embedding dimensions
        ),
    ),
    
    verbose=True,  # Enable detailed logging of crew operations
)
'''
    
    # Print tables to console
    console.print(components_table)
    
    # Display the code configuration example with syntax highlighting
    console.print("\n📝 Configuration Example:")
    console.print(Syntax(code_example, "python", theme="monokai"))

if __name__ == "__main__":
    create_tables()


                                             Memory System Components                                              
╔═══════════════════╦═════════════════════════════════════════════════════════════════════════════════════════════╗
║ Component         ║ Description                                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Short-Term Memory ║ Temporarily stores recent interactions and outcomes using RAG, enabling agents to recall    ║
║                   ║ and utilize information relevant to their current context during executions.                ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Long-Term Memory  ║ Preserves valuable insights and learnings from past executions, allowing agents to build    ║
║                   ║ and refine their knowledge over time.                                                       ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Entity Memory     ║ Captures and organizes information about entities (people, places, concepts) encountered    ║
║                   ║ during tasks, facilitating deeper understanding and relationship mapping. Uses RAG for      ║
║                   ║ storing entity information.                                                                 ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ Contextual Memory ║ Maintains the context of interactions by combining ShortTermMemory, LongTermMemory, and     ║
║                   ║ EntityMemory, aiding in coherence and relevance of agent responses.                         ║
╠═══════════════════╬═════════════════════════════════════════════════════════════════════════════════════════════╣
║ User Memory       ║ Stores user-specific information and preferences, enhancing personalization and user        ║
║                   ║ experience.                                                                                 ║
╚═══════════════════╩═════════════════════════════════════════════════════════════════════════════════════════════╝

📝 Configuration Example:

                                                                                                                   
from crewai import Crew, Agent, Task, Process                                                                      
                                                                                                                   
my_crew = Crew(                                                                                                    
    agents=[...],  # List of AI agents that will work on tasks                                                     
    tasks=[...],   # List of tasks to be executed                                                                  
    process="Process.sequential",  # Tasks will be executed in sequence                                            
    memory=True,   # Enable memory system                                                                          
                                                                                                                   
    # Long-term memory configuration                                                                               
    long_term_memory=EnhanceLongTermMemory(                                                                        
        storage=LTMSQLiteStorage(                                                                                  
            db_path="/my_data_dir/my_crew1/long_term_memory_storage.db"                                            
        )                                                                                                          
    ),                                                                                                             
                                                                                                                   
    # Short-term memory configuration                                                                              
    short_term_memory=EnhanceShortTermMemory(                                                                      
        storage=CustomRAGStorage(                                                                                  
            crew_name="my_crew",           # Unique identifier for the crew                                        
            storage_type="short_term",     # Specifies memory type as short-term                                   
            data_dir="//my_data_dir",      # Directory for storing memory data                                     
            model=embedder["model"],       # Embedding model for RAG                                               
            dimension=embedder["dimension"],# Dimension of embeddings                                              
        ),                                                                                                         
    ),                                                                                                             
                                                                                                                   
    # Entity memory configuration                                                                                  
    entity_memory=EnhanceEntityMemory(                                                                             
        storage=CustomRAGStorage(                                                                                  
            crew_name="my_crew",           # Same crew identifier                                                  
            storage_type="entities",       # Entity-specific storage                                               
            data_dir="//my_data_dir",      # Same data directory                                                   
            model=embedder["model"],       # Same embedding model                                                  
            dimension=embedder["dimension"],# Same embed

## Agent Creation for Contract Analysis Workflow

### Purpose

This Python script defines a function, `create_agents`, which initializes three specialized AI agents designed to process and analyze contract data. Each agent has a unique role in the workflow, equipped with specific tools, goals, and memory capabilities.

---

### Agents Created

1. **PDF Data Extractor Agent**:
   - **Role**: `PDF Data Extractor`
   - **Goal**: Extract key contract terms from PDF files.
   - **Backstory**: An expert in identifying and extracting important contractual information from PDFs.
   - **Tools Used**: `PdfReadTool` for processing PDF content.
   - **Memory**: Enabled for maintaining context during task execution.
   - **LLM**: Uses `ChatOpenAI` with a temperature of 0.7 and the `gpt-4` model for generating context-aware responses.

2. **Data Comparison Analyst Agent**:
   - **Role**: `Data Comparison Analyst`
   - **Goal**: Compare the extracted contract terms against predefined reference values.
   - **Backstory**: A specialist in analyzing and validating contract data for accuracy.
   - **Tools Used**: `ReferenceDataTool` for accessing the standard reference data.
   - **Memory**: Enabled for maintaining contextual continuity during analysis.
   - **LLM**: Utilizes `ChatOpenAI` with a temperature of 0.7 and the `gpt-4` model.

3. **Compliance Report Generator Agent**:
   - **Role**: `Compliance Report Generator`
   - **Goal**: Generate a detailed compliance report highlighting deviations in contract terms.
   - **Backstory**: An expert in drafting compliance documentation based on contract analysis.
   - **Tools Used**: `ReferenceDataTool` for retrieving reference values for comparison.
   - **Memory**: Enabled for efficient report creation and historical context.
   - **LLM**: Powered by `ChatOpenAI` with a temperature of 0.7 and the `gpt-4` model.

---

### Key Features

- **Role Definitions**: Each agent is assigned a distinct role with a clear goal and relevant tools to perform specific tasks in the contract analysis process.
- **Backstory**: Provides a narrative for each agent, enhancing its contextual and operational understanding.
- **Memory**: Ensures that agents can retain and use context throughout their task execution.
- **Tools Integration**: Associates each agent with the necessary tools (`PdfReadTool` or `ReferenceDataTool`) to perform their tasks effectively.
- **Verbose Mode**: Enabled for detailed logging of each agent's operations.
- **Interactive Display**: Utilizes `boxen` to display formatted profiles for each agent, making their roles and goals visually clear.

---

### Workflow

1. **Initialization**: 
   - The `create_agents` function initializes and configures the three agents.
2. **Display Profiles**:
   - Each agent's profile (role, goal, and backstory) is printed in a visually styled box for clarity.
3. **Return Agents**:
   - Returns the three agents: `researcher_agent`, `writer_agent`, and `review_agent` for further use in the workflow.

---

### Example Use Case

This setup is ideal for automating the following contract analysis workflow:
1. Extract terms from a contract using the **PDF Data Extractor Agent**.
2. Validate and compare the terms with predefined reference values using the **Data Comparison Analyst Agent**.
3. Generate a compliance report highlighting any deviations using the **Compliance Report Generator Agent**.


In [15]:
def create_agents():
   # Create PDF Data Extractor Agent
   researcher_agent = Agent(
       role='PDF Data Extractor',
       goal='Extract key contract terms from PDF',
       backstory='Expert in extracting contract terms from PDFs',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[PdfReadTool()],
       memory=True,
   )
   
   # Create Data Comparison Analyst Agent
   writer_agent = Agent(
       role='Data Comparison Analyst',
       goal='Compare contract terms with reference values',
       backstory='Expert in analyzing contract terms',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[ReferenceDataTool()],
       memory=True,
   )
   
   # Create Compliance Report Generator Agent
   review_agent = Agent(
       role='Compliance Report Generator',
       goal='Create detailed report of contract deviations',
       backstory='Expert in compliance reporting',
       llm=ChatOpenAI(temperature=0.7, model='gpt-4'),
       verbose=True,
       tools=[ReferenceDataTool()],
       memory=True,
   )
   
   # Display formatted information for each agent
   for agent in [researcher_agent, writer_agent, review_agent]:
       print(boxen(
           f"Role: {agent.role}\nGoal: {agent.goal}\nBackstory: {agent.backstory}",
           title="🤖 Agent Profile",
           padding=1,
           margin=1,
           color="green"
       ))
   
   return researcher_agent, writer_agent, review_agent

# Initialize all agents
researcher_agent, writer_agent, review_agent = create_agents()

c:\Users\sachi\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


                                                                                                                   
   ╭─ 🤖 Agent Profile ───────────────────────────────────────────╮                                                
   │                                                              │                                                
   │   Role: PDF Data Extractor                                   │                                                
   │   Goal: Extract key contract terms from PDF                  │                                                
   │   Backstory: Expert in extracting contract terms from PDFs   │                                                
   │                                                              │                                                
   ╰──────────────────────────────────────────────────────────────╯                                                
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ─────────────────────────────────────╮                                                      
   │                                                        │                                                      
   │   Role: Data Comparison Analyst                        │                                                      
   │   Goal: Compare contract terms with reference values   │                                                      
   │   Backstory: Expert in analyzing contract terms        │                                                      
   │                                                        │                                                      
   ╰────────────────────────────────────────────────────────╯                                                      
                                                                         

                                                                                                                   
   ╭─ 🤖 Agent Profile ──────────────────────────────────────╮                                                     
   │                                                         │                                                     
   │   Role: Compliance Report Generator                     │                                                     
   │   Goal: Create detailed report of contract deviations   │                                                     
   │   Backstory: Expert in compliance reporting             │                                                     
   │                                                         │                                                     
   ╰─────────────────────────────────────────────────────────╯                                                     
                                                                         

## Task Creation for Contract Analysis Workflow

### Purpose

This Python script defines a function, `create_tasks`, which creates and initializes tasks for an automated contract analysis workflow. Each task is assigned to a specific agent, ensuring a structured and efficient process from data extraction to compliance reporting.

---

### Tasks Created

1. **PDF Data Extraction Task**:
   - **Description**: 
     - Uses the `PDF Reader` tool to process the input file and extract key contract terms.
   - **Assigned Agent**: `researcher_agent` (PDF Data Extractor).
   - **Expected Output**: A dictionary containing the extracted contract terms.

2. **Data Comparison Task**:
   - **Description**:
     - Compares the extracted terms against predefined reference data to identify mismatches.
   - **Assigned Agent**: `writer_agent` (Data Comparison Analyst).
   - **Expected Output**: A list of mismatches between the extracted data and the reference values.

3. **Report Generation Task**:
   - **Description**:
     - Creates a detailed analysis report summarizing the findings and recommendations.
   - **Assigned Agent**: `review_agent` (Compliance Report Generator).
   - **Expected Output**: A comprehensive report containing findings and actionable insights.

---

### Key Features

- **Task Definition**:
  - Each task is clearly defined with a detailed description, specifying its objective and expected output.
- **Agent Assignment**:
  - Tasks are assigned to the appropriate agent based on their role and expertise, ensuring alignment with the workflow goals.
- **Expected Outputs**:
  - Clearly defines the deliverable for each task, providing a measure of completion and success.
- **Dynamic Input Handling**:
  - Accepts the `input_file` parameter, making the script reusable for analyzing different PDF files.

---

### Workflow

1. **PDF Data Extraction**:
   - The first task involves reading the PDF file and extracting key terms using the `PDF Reader` tool.
2. **Data Comparison**:
   - The second task compares the extracted data with reference values to identify mismatches.
3. **Report Generation**:
   - The third task generates a comprehensive compliance report summarizing findings and deviations.

---

### Interactive Display

- The `boxen` utility is used to print formatted details of each task, making it visually clear and easy to understand.

---

### Example Use Case

This script is part of a larger automated workflow for contract analysis:
1. Input a contract PDF file (`contract.pdf`) into the system.
2. Execute the tasks sequentially:
   - Extract key terms from the PDF.
   - Validate and compare the terms against reference data.
   - Generate a detailed compliance report.

---

### Code Initialization

The function is called with:
- **Agents**: `researcher_agent`, `writer_agent`, `review_agent` (previously initialized).
- **Input File**: `contract.pdf`.

It returns the created tasks for further processing in the workflow.


In [21]:
from crewai import Task

def create_tasks(researcher_agent, writer_agent, review_agent, input_file):
   # Import textwrap for clean multiline strings
   from textwrap import dedent
   
   # Task 1: PDF Data Extraction
   pdf_extraction_task = Task(
       description=dedent(f"""Use the PDF Reader tool to read {input_file} and extract key terms."""),
       agent=researcher_agent,
       expected_output="Dictionary containing extracted contract terms"
   )
   
   # Task 2: Data Comparison
   comparison_task = Task(
       description=dedent(f"""Compare extracted terms with reference data and identify mismatches."""),
       agent=writer_agent,
       expected_output="List of mismatches between extracted and reference data"
   )
   
   # Task 3: Report Generation
   final_report_task = Task(
       description=dedent(f"""Create comprehensive analysis report with findings."""),
       agent=review_agent,
       expected_output="Detailed analysis report with findings and recommendations"
   )
   
   # Display formatted information for each task
   for task in [pdf_extraction_task, comparison_task, final_report_task]:
       print(boxen(
           f"Description: {task.description}\nExpected Output: {task.expected_output}",
           title="📝 Task Details",
           padding=1,
           margin=1,
           color="yellow"
       ))
   
   return pdf_extraction_task, comparison_task, final_report_task

# Initialize tasks with input file
input_file = "contract.pdf"
tasks = create_tasks(researcher_agent, writer_agent, review_agent, input_file)

                                                                                                                   
   ╭─ 📝 Task Details ────────────────────────────────────────────────────────────────────╮                        
   │                                                                                      │                        
   │   Description: Use the PDF Reader tool to read contract.pdf and extract key terms.   │                        
   │   Expected Output: Dictionary containing extracted contract terms                    │                        
   │                                                                                      │                        
   ╰──────────────────────────────────────────────────────────────────────────────────────╯                        
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ─────────────────────────────────────────────────────────────────────╮                       
   │                                                                                       │                       
   │   Description: Compare extracted terms with reference data and identify mismatches.   │                       
   │   Expected Output: List of mismatches between extracted and reference data            │                       
   │                                                                                       │                       
   ╰───────────────────────────────────────────────────────────────────────────────────────╯                       
                                                                                                                   



                                                                                                                   
   ╭─ 📝 Task Details ───────────────────────────────────────────────────────────────╮                             
   │                                                                                 │                             
   │   Description: Create comprehensive analysis report with findings.              │                             
   │   Expected Output: Detailed analysis report with findings and recommendations   │                             
   │                                                                                 │                             
   ╰─────────────────────────────────────────────────────────────────────────────────╯                             
                                                                                                                   



# Contract Analysis System with CrewAI

## Purpose

This script sets up and runs the complete contract analysis workflow using `CrewAI`. The system integrates multiple agents and tasks to process a contract PDF file, compare its terms with reference data, and generate a compliance report of mismatch data.

---

## Function: `run_analysis`

### Description

The `run_analysis` function orchestrates the workflow by:

1. **Initializing the Crew**:
   - Combines the provided agents and tasks into a `Crew` instance.
   - Enables verbose mode for detailed logging of the process.

2. **Executing the Workflow**:
   - Calls the `kickoff()` method to begin processing the tasks sequentially.

3. **Displaying Results**:
   - Displays the final output (analysis report) in a visually appealing format using the `boxen` utility.

---

### Parameters

- **agents**:
  - A list of agents involved in the workflow:
    - `researcher_agent`: Extracts data from the PDF.
    - `writer_agent`: Compares extracted data with reference values.
    - `review_agent`: Generates the final compliance report.

- **tasks**:
  - A list of tasks to be executed sequentially:
    - PDF data extraction, data comparison, and report generation.

---

### Output

- **Final Report**:
  - Give you the final report of the mismatch data of the PDF with the refrence data we have
  - The result of the `kickoff()` method contains the analysis report.
  - The report is displayed with a visually formatted box, making it easy to interpret.

---

## Workflow

1. **Input**:
   - Agents: `[researcher_agent, writer_agent, review_agent]`.
   - Tasks: `[pdf_extraction_task, comparison_task, final_report_task]`.

2. **Execution**:
   - The `Crew` processes the tasks sequentially, delegating each task to its assigned agent.

3. **Output**:
   - A detailed compliance report summarizing the findings and recommendations.

---

## Example Use Case

This code is part of a contract analysis system. The workflow follows these steps:

1. **Initialization**:
   - Define agents and tasks required for the analysis.
2. **Execution**:
   - Run the analysis using the `run_analysis` function.
3. **Result**:
   - Receive a comprehensive compliance report with actionable insights.

---

## Display

The result is presented using the `boxen` utility:
- **Title**: `📊 Contract Analysis Report`.
- **Formatting**:
  - Padding and margin for clear separation.
  - Color-coded for emphasis (magenta).

---

## Example Invocation

```python
final_report = run_analysis([researcher_agent, writer_agent, review_agent], tasks)


In [25]:
# Contract Analysis System with CrewAI
def run_analysis(agents, tasks):
    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=True
    )
    
    result = crew.kickoff()
    
    # Display final results with supported parameters only
    print(boxen(
        result,
        title="📊 Contract Analysis Report",
        padding=1,
        margin=1,
        color="magenta"
    ))
    
    return result

# Execute analysis
final_report = run_analysis([researcher_agent, writer_agent, review_agent], tasks)


2024-12-24 21:06:11,524 - 22992 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: PDF Data Extractor
 [INFO]: == Starting Task: Use the PDF Reader tool to read contract.pdf and extract key terms.


> Entering new CrewAgentExecutor chain...
First, I need to use the PDF Reader tool to read the contract.pdf and extract the key terms.

Action: 
PDF Reader

Action Input: 
{
  "file": "contract.pdf"
}
 

CONSULTING SERVICES AGREEMENT  
 
This Consulting Services Agreement ("Agreement") is made effective as of 15/01/2024, by and between:  
 
ABC Technologies Inc.  
123 Tech Park Avenue  
Silicon Valley, CA 94025  
 
Contact: john.doe@abctech.com  
 
and 
 
XYZ Consulting LLC ("Consultant")  
456 Business Plaza  
New York, NY 10001  
 
KEY TERMS AND CONDITIONS:  
 
1. Agreement Duration:  
   Start Date: 15/01/2024  
   End Date: 14/01/2025  
 
2. Service Scope:  
   - Technical consulting  
   - Project management  
   - Strategic advisory  
 
3. Financial Terms:  
   - Rate: $150 per hour     - Payment terms: Net 45 days  
   - Maximum monthly 

                                                                                                                   
   ╭─ 📊 Contract Analysis Report ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Compliance Report:                                                                                      │   
   │                                                                                                           │   
   │   Title: Analysis of Contract Deviations                                                                  │   
   │                                                                                                           │   
   │   Introduction:                                                                                           │   
   │   This report presents an analysis of the discrepancies identified b